In [67]:
import pandas as pd
import numpy as np
from helper import get_mapper
import json

In [68]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")

In [69]:
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [71]:
CDF = pd.read_csv("CDFt1.csv", parse_dates=['produced_at'])

In [72]:
powers = pd.read_csv("CDF2t1.csv", parse_dates=['produced_at'])

In [73]:
powers2 = powers.copy()
powers2['power'] = powers2['power'].fillna(0)
#powers2.power.replace(['-'], [0], inplace=True)
powers3 = powers2.astype({"power": int})
#powers3.to_csv("produced_power_pg.csv", index=True, header=False)
#powers3.to_csv("produced_power.csv", index=False)
#powers3.to_csv("produced_power_nh.csv", index=False, header=False)

In [74]:
plist = list(CDF)

In [75]:
"BNA0997" in plist

False

In [76]:
years = ("fix/Gaskraftwerk_Irsching_201701010000_201512312345_97.csv", "fix/Gaskraftwerk_Irsching_201601010000_201712312345_97.csv", "fix/Gaskraftwerk_Irsching_201801010000_201912312345_97.csv")

In [77]:
test = pd.read_csv(years[2])

In [78]:
test.describe()

,BNA0995,BNA0993,BNA0994
count,17520.000000,17520.000000,17520.000000
mean,0.628995,0.550228,2.346575
std,14.333746,10.034844,30.907384
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,562.000000,360.000000,781.000000


In [79]:
gy = CDF.resample('1Y', on='produced_at').sum()
gm = CDF.resample('1M', on='produced_at').sum()

In [80]:
gm2 = gm.reset_index()
gm2['year'] = gm2['produced_at']
gm2['month'] = gm2['produced_at']

In [81]:
gm2['year'] = gm2['year'].apply(lambda x: str(x).split("-")[0])
gm2['month'] = gm2['month'].apply(lambda x: int(str(x).split("-")[1]))
gm2['month'] = gm2['month'].astype(int)

In [82]:
gm3 = gm2.sort_values(by=["year", 'month'])
gm4 = gm3.drop(columns='produced_at')

In [83]:
gm5 = gm4.melt(id_vars=["year", "month"], var_name='blockid', value_name='power')
gm5['power'] = gm5['power'].astype(int)

In [84]:
gm5

,year,month,blockid,power
0,2015,1,BNA0067,70621
1,2015,2,BNA0067,68088
2,2015,3,BNA0067,66709
3,2015,4,BNA0067,53328
4,2015,5,BNA0067,76938
...,...,...,...,...
15295,2019,8,BNA1093,58135
15296,2019,9,BNA1093,58514
15297,2019,10,BNA1093,110960
15298,2019,11,BNA1093,151811


In [85]:
#gm5.to_csv("monthly.csv", header=False)

In [86]:
gm5.describe()

,month,power
count,15300.000000,1.530000e+04
mean,6.500000,9.481408e+04
std,3.452165,1.751555e+05
min,1.000000,0.000000e+00
25%,3.750000,0.000000e+00
50%,6.500000,1.175950e+04
75%,9.250000,1.147488e+05
max,12.000000,1.028670e+06


In [87]:
pm1 = gm5.merge(bpm, on="blockid")

In [88]:
gm6 = gm5.copy()

In [89]:
gm6 = gm5.groupby(by=['blockid', 'month', 'year'])

In [90]:
m = pm1.duplicated(subset=['year', 'month', 'plantid'])

In [91]:
pm1.loc[m]

,year,month,blockid,power,plantid
120,2015,1,BNA0124,532722,14-70-46630660001
121,2015,2,BNA0124,496284,14-70-46630660001
122,2015,3,BNA0124,507962,14-70-46630660001
123,2015,4,BNA0124,541273,14-70-46630660001
124,2015,5,BNA0124,576302,14-70-46630660001
...,...,...,...,...,...
13195,2019,8,BNA1093,58135,06-09-100-0003-0003
13196,2019,9,BNA1093,58514,06-09-100-0003-0003
13197,2019,10,BNA1093,110960,06-09-100-0003-0003
13198,2019,11,BNA1093,151811,06-09-100-0003-0003


In [92]:
pm2 = pm1.copy()

In [93]:
pm2['plantpower'] = pm2.groupby(['plantid', 'month', 'year'])['power'].transform('sum')

In [94]:
pm3 = pm2.drop_duplicates(['month', 'year', 'plantid'])

In [95]:
pm3.sort_values(by="plantpower", ascending=False)

,year,month,blockid,power,plantid,plantpower
9523,2018,8,BNA1401b,641439,06-05-100-0248923,2633329
9501,2016,10,BNA1401b,723383,06-05-100-0248923,2631880
9525,2018,10,BNA1401b,688779,06-05-100-0248923,2608109
9522,2018,7,BNA1401b,609273,06-05-100-0248923,2577498
9518,2018,3,BNA1401b,702196,06-05-100-0248923,2557329
...,...,...,...,...,...,...
5498,2018,3,BNA0290,0,06-00464010412,0
5499,2018,4,BNA0290,0,06-00464010412,0
5500,2018,5,BNA0290,0,06-00464010412,0
5501,2018,6,BNA0290,0,06-00464010412,0


In [96]:
pm4 = pm3.drop(['blockid', 'power'], axis=1)

In [97]:
pm4

,year,month,plantid,plantpower
0,2015,1,06-05-900-0205103,70621
1,2015,2,06-05-900-0205103,68088
2,2015,3,06-05-900-0205103,66709
3,2015,4,06-05-900-0205103,53328
4,2015,5,06-05-900-0205103,76938
...,...,...,...,...
13135,2019,8,06-09-100-0003-0003,58135
13136,2019,9,06-09-100-0003-0003,58514
13137,2019,10,06-09-100-0003-0003,110960
13138,2019,11,06-09-100-0003-0003,151811


In [98]:
pm4.to_csv("monthlyp_pg.csv", header=False)

In [99]:
pm4.loc[pm4.plantid == "06-04-11_2039669_2_0"]

,year,month,plantid,plantpower
1500,2015,1,06-04-11_2039669_2_0,173077
1501,2015,2,06-04-11_2039669_2_0,157209
1502,2015,3,06-04-11_2039669_2_0,148299
1503,2015,4,06-04-11_2039669_2_0,137687
1504,2015,5,06-04-11_2039669_2_0,95119
1505,2015,6,06-04-11_2039669_2_0,124325
1506,2015,7,06-04-11_2039669_2_0,150438
1507,2015,8,06-04-11_2039669_2_0,160772
1508,2015,9,06-04-11_2039669_2_0,138932
1509,2015,10,06-04-11_2039669_2_0,176989
